# One-to-Many and Many-to-Many Joins - Lab

## Introduction

In this lab, you'll practice your knowledge of one-to-many and many-to-many relationships!

## Objectives

You will be able to:

* Explain one-to-many and many-to-many joins as well as implications for the size of query results
* Query data using one-to-many and many-to-many joins

## One-to-Many and Many-to-Many Joins
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connect to the Database

Include the relevant imports, then connect to the database located at `data.sqlite`.

In [3]:
# Your code here
import sqlite3
import pandas as pd
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Employees and Their Offices (a One-to-One Join)

Select all of the employees including their first name and last name along with the city and state of the office that they work out of (if they have one). Include all employees and order them by their first name, then their last name.

In [8]:
# Your code here
df = pd.read_sql(''' SELECT employees.firstName AS first_name,
                  employees.lastName AS last_name,
                 offices.city, offices.state FROM employees
                 JOIN offices
                 ON employees.officeCode = offices.officeCode
                 ORDER BY first_name;''', conn )
print(len(df))
print(df)

23
   first_name  last_name           city       state
0        Andy     Fixter         Sydney            
1     Anthony        Bow  San Francisco          CA
2       Barry      Jones         London            
3       Diane     Murphy  San Francisco          CA
4    Foon Yue      Tseng            NYC          NY
5      George     Vanauf            NYC          NY
6      Gerard     Bondur          Paris            
7      Gerard  Hernandez          Paris            
8        Jeff   Firrelli  San Francisco          CA
9       Julie   Firrelli         Boston          MA
10      Larry       Bott         London            
11     Leslie   Jennings  San Francisco          CA
12     Leslie   Thompson  San Francisco          CA
13       Loui     Bondur          Paris            
14       Mami      Nishi          Tokyo  Chiyoda-Ku
15     Martin     Gerard          Paris            
16       Mary  Patterson  San Francisco          CA
17     Pamela   Castillo          Paris            
18      P

## Customers and Their Orders (a One-to-Many Join)

Select all of the customer contacts (first and last names) along with details for each of the customers' order numbers, order dates, and statuses.

In [19]:
# Your code here
df = pd.read_sql('''SELECT customers.contactFirstName AS first_name, customers.contactLastName AS last_name,
                  orders.orderNumber,orders.orderDate, orders.status FROM customers
                 JOIN orders
                 ON customers.customerNumber = orders.customerNumber
                 WHERE status != 'Shipped'
                 GROUP BY contactFirstName
                 HAVING orderNumber >= 10414 ;''',conn)
df

,first_name,last_name,orderNumber,orderDate,status
0,Adrian,Huxley,10420,2005-05-29,In Process
1,Catherine,Dewey,10423,2005-05-30,In Process
2,Janine,Labrune,10425,2005-05-31,In Process
3,Juri,Yoshido,10414,2005-05-06,On Hold
4,Kelvin,Leong,10422,2005-05-30,In Process
5,Sean,Clenahan,10415,2005-05-09,Disputed
6,Susan,Nelson,10421,2005-05-29,In Process


## Customers and Their Payments (Another One-to-Many Join)

Select all of the customer contacts (first and last names) along with details for each of the customers' payment amounts and date of payment. Sort these results in descending order by the payment amount. 

In [22]:
# Your code here
df = pd.read_sql(''' SELECT customers.contactFirstName as first_name, customers.contactLastName as last_name, 
                 payments.paymentDate as payment_date, payments.amount 
                 FROM customers
                 JOIN payments
                 USING (customerNumber)
                 ORDER BY amount DESC;''',conn)
df

,first_name,last_name,payment_date,amount
0,Diego,Freyre,2005-03-18,120166.58
1,Diego,Freyre,2004-12-31,116208.40
2,Susan,Nelson,2003-08-15,111654.40
3,Eric,Natividad,2003-12-26,105743.00
4,Susan,Nelson,2005-03-05,101244.59
...,...,...,...,...
268,Carine,Schmitt,2004-12-18,1676.14
269,Pascale,Cartrain,2003-04-19,1627.56
270,Jonas,Bergulfsen,2003-10-28,1491.38
271,Pascale,Cartrain,2003-08-22,1128.20


## Orders, Order Details, and Product Details (a Many-to-Many Join)

Select all of the customer contacts (first and last names) along with the product names, quantities, and date ordered for each of the customers and each of their orders. Sort these in descending order by the order date.

> Note: This will require joining 4 tables! This can be tricky! Give it a shot, and if you're still stuck, turn to the next section where you'll see how to write subqueries that can make complex queries such as this much simpler!

In [35]:
# Your code here
df = pd.read_sql(''' 
                 SELECT 
                 customers.contactFirstName AS first_name,
                 customers.contactLastName as last_name,
                 products.productName, 
                 products.quantityinStock,
                 orders.orderDate,
                 orderdetails.orderNumber
                 FROM customers
                 JOIN orders ON 
                 customers.customerNumber = orders.customerNumber
                 JOIN orderdetails ON
                 orders.orderNumber = orderdetails.orderNumber
                 JOIN products ON
                 orderdetails.productCode = products.productCode
                 ORDER BY quantityInStock DESC
                 ;''',conn )
df

,first_name,last_name,productName,quantityInStock,orderDate,orderNumber
0,Janine,Labrune,2002 Suzuki XREO,9997,2005-02-03,10375
1,Jonas,Bergulfsen,2002 Suzuki XREO,9997,2004-10-15,10309
2,Julie,Murphy,2002 Suzuki XREO,9997,2003-12-01,10201
3,Kwai,Lee,2002 Suzuki XREO,9997,2003-02-24,10107
4,Kwai,Lee,2002 Suzuki XREO,9997,2004-11-15,10329
...,...,...,...,...,...,...
2991,Dominique,Perrier,1960 BSA Gold Star DBD34,15,2005-04-07,10402
2992,Martha,Larsson,1960 BSA Gold Star DBD34,15,2005-03-03,10389
2993,Sue,Frick,1960 BSA Gold Star DBD34,15,2004-05-11,10250
2994,Rosa,Salazar,1960 BSA Gold Star DBD34,15,2004-04-03,10236


## Summary

In this lab, you practiced your knowledge of one-to-many and many-to-many relationships!